In [1]:
!nvidia-smi

Wed May 18 04:47:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q transformers
!pip install emoji==0.6.0

In [3]:
#transformers
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler 
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForSequenceClassification

In [4]:
df = pd.read_csv('/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/clean_train.csv', header=0)
df.head()

,text_clean,Sentiment
0,it s a confusing odd time for the shopping pub...,Negative
1,in 2019 d2c ecommerce sales reached 1428 billi...,Positive
2,chinese residents are paying exorbitant prices...,Negative
3,list of supermarkets grocery shops and vegetab...,Neutral
4,its there any wonder tesco and other supermark...,Positive


In [5]:
df_test = pd.read_csv('/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/clean_test.csv', header=0)
df_test.head()

,text_clean,Sentiment
0,we may not have any toilet paper in our house ...,Positive
1,really whats the downside of coronavirus for a...,Positive
2,hello everyone we made amp sell high quality m...,Extremely Positive
3,happy to report that i jumped on the panic sho...,Positive
4,just been to the supermarket why do all women ...,Neutral


# Sentiment column ananlysis

In [6]:
df['Sentiment'].value_counts()

Positive              11381
Negative               9889
Neutral                7560
Extremely Positive     6618
Extremely Negative     5475
Name: Sentiment, dtype: int64

In [7]:
df['Sentiment'] = df['Sentiment'].map({'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1,
                                       'Positive': 2, 'Extremely Positive': 2})

In [8]:
df_test['Sentiment'] = df_test['Sentiment'].map({'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1,
                                                 'Positive': 2, 'Extremely Positive': 2})

In [9]:
df['Sentiment'].value_counts()

2    17999
0    15364
1     7560
Name: Sentiment, dtype: int64

## Class Balanceing by RandomOverSampler

In [10]:
# ros = RandomOverSampler()
# train_x, train_y = ros.fit_resample(np.array(df['text_clean']).reshape(-1, 1), np.array(df['Sentiment']).reshape(-1, 1))
# train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns=['text_clean', 'Sentiment'])

In [11]:
train_os = df

In [12]:
train_os['Sentiment'].value_counts()

2    17999
0    15364
1     7560
Name: Sentiment, dtype: int64

## Train - Validation - Test split

In [13]:
train_texts = train_os['text_clean'].values.tolist()
train_labels = train_os['Sentiment'].values.tolist()

In [14]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, stratify=train_labels, random_state=42)

In [15]:
test_texts = df_test['text_clean'].values.tolist()
test_labels = df_test['Sentiment'].values.tolist()

In [16]:
print(f"TRAINING DATA: {len(train_texts)}\nVALIDATION DATA: {len(val_texts)}\nTESTING DATA: {len(test_texts)}" )

TRAINING DATA: 32738
VALIDATION DATA: 8185
TESTING DATA: 3787


# roBERTa Sentiment analysis

In [17]:
MAX_LEN=128

In [18]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
train_encodings = tokenizer(train_texts, max_length=MAX_LEN,
                            padding='max_length', truncation=True)
val_encodings = tokenizer(val_texts, max_length=MAX_LEN,
                            padding='max_length', truncation=True)
test_encodings = tokenizer(test_texts, max_length=MAX_LEN,
                            padding='max_length', truncation=True)

In [20]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
test_dataset = TweetDataset(test_encodings, test_labels)

# Fine-tuning with native PyTorch/TensorFlow

In [21]:
# config = AutoConfig.from_pretrained("vinai/bertweet-base", num_labels=3)
# model = AutoModelForSequenceClassification.from_config(config)

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=3)

model

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [22]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [23]:
from tqdm.notebook import tqdm

def train_model(model, train_dataloader, val_dataloader, epochs=20, device='cuda'):
    model.to(device)
    
    optimizer = AdamW(model.parameters(), lr=3e-6)

    for epoch_num in range(epochs):
        model.train()
        for batch in tqdm(train_dataloader):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()
        
        total_val_loss = 0
        total_val_acc = 0
        model.eval()

        with torch.no_grad():
            for batch in tqdm(val_dataloader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

                loss = outputs.loss
                logits = outputs.logits

                total_val_loss += loss.item()
                total_val_acc += (logits.argmax(axis=1) == labels).sum().item()
        
        print("Epoch {} ---------------------------------".format(epoch_num))
        print("Validation loss: {}".format(total_val_loss / (128 * len(val_dataloader))))
        print("Validation acc: {}".format(total_val_acc / (128 * len(val_dataloader))))


train_model(model, train_dataloader, val_dataloader, 30)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 0 ---------------------------------
Validation loss: 0.006331366254016757
Validation acc: 0.657470703125


  0%|          | 0/256 [00:00<?, ?it/s]

...

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 28 ---------------------------------
Validation loss: 0.0030673262826894643
Validation acc: 0.8955078125


  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 29 ---------------------------------
Validation loss: 0.0031287602614611387
Validation acc: 0.8978271484375


# roBERTa result

In [24]:
def test_model(model, test_dataloader, device='cuda'):
    model.to(device)
    model.eval()

    total_test_loss = 0
    total_test_acc = 0

    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss
            logits = outputs.logits

            total_test_loss += loss.item()
            total_test_acc += (logits.argmax(axis=1) == labels).sum().item()
    
    print("Test acc: {}".format(total_test_acc / (128 * len(test_dataloader))))


test_model(model, test_dataloader)

  0%|          | 0/30 [00:00<?, ?it/s]

Test acc: 0.8770833333333333
